## 🔧 第一步：环境信息检查

在开始部署模型之前，我们需要了解当前的运行环境。这个步骤非常重要，因为：

### 🎯 检查目的
1. **硬件确认**: 确保有足够的 GPU 显存运行模型
2. **系统兼容**: 验证操作系统和 Python 版本
3. **资源评估**: 了解可用的 CPU、内存和存储空间
4. **环境配置**: 检查 CUDA 版本和相关依赖

### 📊 检查内容
- **操作系统**: Linux 发行版和版本
- **CPU 信息**: 处理器型号和核心数
- **内存状态**: 总内存和可用内存
- **GPU 配置**: 显卡型号和显存大小
- **CUDA 版本**: 深度学习框架支持
- **Python 环境**: 解释器版本
- **磁盘空间**: 可用存储空间

In [2]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助您：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
!pip install pandas==2.2.2

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引

### 环境信息
| 项目         | 信息                                                               |
|:-------------|:-------------------------------------------------------------------|
| 操作系统     | Linux Ubuntu 22.04.4 LTS                                           |
| CPU 信息     | Intel(R) Xeon(R) CPU @ 2.20GHz (1 physical cores, 2 logical cores) |
| 内存信息     | 12.67 GB (Available: 10.44 GB)                                     |
| GPU 信息     | No GPU found (nvidia-smi not found)                                |
| CUDA 信息    | 12.5                                                               |
| Python 版本  | 3.11.13                                                            |
| Conda 版本   | Conda not found                                                    |
| 物理磁盘空间 | Total: 107.72 GB, Used: 37.81 GB, Free: 69.88 GB                   |


## 📦 第二步：安装依赖包

现在我们需要安装运行模型所需的关键 Python 包：

In [3]:
# Transformers 安装
! pip install transformers datasets evaluate accelerate
# 如果要从源码安装而不是最新发布版本，请注释上面的命令并取消注释下面的命令。
# ! pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

# 文本生成

文本生成是大语言模型（LLMs）最受欢迎的应用。大语言模型经过训练，能够根据给定的初始文本（提示词）以及其自身生成的输出，生成下一个词（token），直到达到预定义的长度或遇到序列结束（`EOS`）token。

在 Transformers 中，[generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) API 负责处理文本生成，它适用于所有具有生成能力的模型。本指南将向您展示使用 [generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) 进行文本生成的基础知识以及一些需要避免的常见陷阱。


> [!TIP] 您也可以直接从命令行与模型对话。([参考文档](https://huggingface.co/docs/transformers/main/en/./conversations.md#transformers-cli)) ``` transformers chat Qwen/Qwen2.5-0.5B-Instruct```

## 默认生成

在开始之前，建议安装 [bitsandbytes](https://hf.co/docs/bitsandbytes/index) 来量化超大模型以减少内存使用。\n\n```bash\n!pip install -U transformers bitsandbytes\n```\n除了基于 CUDA 的 GPU 外，Bitsandbytes 还支持多种后端。请参考多后端安装[指南](https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend)了解更多信息。\n\n使用 [from_pretrained()](https://huggingface.co/docs/transformers/main/en/main_classes/model#transformers.PreTrainedModel.from_pretrained) 加载大语言模型，并添加以下两个参数来减少内存需求。\n\n- `device_map=\"auto\"` 启用 Accelerate 的[大模型推理](https://huggingface.co/docs/transformers/main/en/./models#big-model-inference)功能，用于自动初始化模型骨架并在所有可用设备上加载和分发模型权重，从最快的设备（GPU）开始。\n- `quantization_config` 是定义量化设置的配置对象。此示例使用 bitsandbytes 作为量化后端（有关更多可用后端，请参见[量化](https://huggingface.co/docs/transformers/main/en/./quantization/overview)部分），并以 [4位](https://huggingface.co/docs/transformers/main/en/./quantization/bitsandbytes)精度加载模型。

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1",device_map="auto",quantization_config=quantization_config)

对输入进行分词，并将 `padding_side()` 参数设置为 `\"left\"`，因为大语言模型没有被训练从填充token继续生成。分词器返回输入ID和注意力掩码。\n\n> [!TIP]\n> 通过向分词器传递字符串列表，可以一次处理多个提示词。批量处理输入可以提高吞吐量，但会略微增加延迟和内存消耗。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(\"mistralai/Mistral-7B-v0.1\", padding_side=\"left\")\nmodel_inputs = tokenizer([\"A list of colors: red, blue\"], return_tensors=\"pt\").to(\"cuda\")

将输入传递给 [generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) 来生成token，并使用 [batch_decode()](https://huggingface.co/docs/transformers/main/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.batch_decode) 将生成的token解码回文本。

In [ ]:
generated_ids = model.generate(**model_inputs)\ntokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]\n\"A list of colors: red, blue, green, yellow, orange, purple, pink,\"

## 生成配置

所有生成设置都包含在 [GenerationConfig](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig) 中。在上面的示例中，生成设置来自 [mistralai/Mistral-7B-v0.1](https://huggingface.co/mistralai/Mistral-7B-v0.1) 的 `generation_config.json` 文件。当模型没有保存配置时，会使用默认的解码策略。\n\n通过 `generation_config` 属性检查配置。它只显示与默认配置不同的值，在这种情况下是 `bos_token_id` 和 `eos_token_id`。

In [ ]:
from transformers import AutoModelForCausalLM\n\nmodel = AutoModelForCausalLM.from_pretrained(\"mistralai/Mistral-7B-v0.1\", device_map=\"auto\")\nmodel.generation_config\nGenerationConfig {\n  \"bos_token_id\": 1,\n  \"eos_token_id\": 2\n}

您可以通过覆盖 [GenerationConfig](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig) 中的参数和值来自定义 [generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate)。有关常用调整参数，请参见[下面的部分](#common-options)。

In [ ]:
# 启用束搜索采样策略\nmodel.generate(**model_inputs, num_beams=4, do_sample=True)

[generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) 也可以通过外部库或自定义代码进行扩展：\n1. `logits_processor` 参数接受自定义的 [LogitsProcessor](https://huggingface.co/docs/transformers/main/en/internal/generation_utils#transformers.LogitsProcessor) 实例，用于操作下一个token的概率分布；\n2. `stopping_criteria` 参数支持自定义的 [StoppingCriteria](https://huggingface.co/docs/transformers/main/en/internal/generation_utils#transformers.StoppingCriteria) 来停止文本生成；\n3. 其他自定义生成方法可以通过 `custom_generate` 标志加载（[文档](https://huggingface.co/docs/transformers/main/en/generation_strategies.md/#custom-decoding-methods)）。\n\n请参考[生成策略](https://huggingface.co/docs/transformers/main/en/./generation_strategies)指南，了解更多关于搜索、采样和解码策略的信息。

### 保存

创建一个 [GenerationConfig](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig) 实例并指定您想要的解码参数。

In [ ]:
from transformers import AutoModelForCausalLM, GenerationConfig\n\nmodel = AutoModelForCausalLM.from_pretrained(\"my_account/my_model\")\ngeneration_config = GenerationConfig(\n    max_new_tokens=50, do_sample=True, top_k=50, eos_token_id=model.config.eos_token_id\n)

使用 [save_pretrained()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig.save_pretrained) 保存特定的生成配置，并将 `push_to_hub` 参数设置为 `True` 以将其上传到 Hub。

In [ ]:
generation_config.save_pretrained(\"my_account/my_model\", push_to_hub=True)

将 `config_file_name` 参数留空。当在单个目录中存储多个生成配置时，应使用此参数。它为您提供了一种指定要加载哪个生成配置的方法。您可以为不同的生成任务创建不同的配置（使用采样的创意文本生成、使用束搜索的摘要生成），以便与单个模型一起使用。

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig\n\ntokenizer = AutoTokenizer.from_pretrained(\"google-t5/t5-small\")\nmodel = AutoModelForSeq2SeqLM.from_pretrained(\"google-t5/t5-small\")\n\ntranslation_generation_config = GenerationConfig(\n    num_beams=4,\n    early_stopping=True,\n    decoder_start_token_id=0,\n    eos_token_id=model.config.eos_token_id,\n    pad_token=model.config.pad_token_id,\n)\n\ntranslation_generation_config.save_pretrained(\"/tmp\", config_file_name=\"translation_generation_config.json\", push_to_hub=True)\n\ngeneration_config = GenerationConfig.from_pretrained(\"/tmp\", config_file_name=\"translation_generation_config.json\")\ninputs = tokenizer(\"translate English to French: Configuration files are easy to use!\", return_tensors=\"pt\")\noutputs = model.generate(**inputs, generation_config=generation_config)\nprint(tokenizer.batch_decode(outputs, skip_special_tokens=True))

## 常用选项

[generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) 是一个功能强大且可高度自定义的工具。这对新用户来说可能会感到困惑。本节包含了您可以在 Transformers 的大多数文本生成工具中定义的常用生成选项列表：[generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate)、[GenerationConfig](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig)、`pipelines`、`chat` CLI 等...\n\n| 选项名称 | 类型 | 简化描述 |\n|---|---|---|\n| `max_new_tokens` | `int` | 控制最大生成长度。请务必定义它，因为它通常默认为一个较小的值。 |\n| `do_sample` | `bool` | 定义生成是否会采样下一个token（`True`），或者使用贪心策略（`False`）。大多数用例应将此标志设置为 `True`。查看[此指南](https://huggingface.co/docs/transformers/main/en/./generation_strategies.md)了解更多信息。 |\n| `temperature` | `float` | 下一个选择的token的不可预测性。高值（`>0.8`）适合创意任务，低值（例如 `<0.4`）适合需要\"思考\"的任务。需要 `do_sample=True`。 |\n| `num_beams` | `int` | 当设置为 `>1` 时，激活束搜索算法。束搜索在基于输入的任务上表现良好。查看[此指南](https://huggingface.co/docs/transformers/main/en/./generation_strategies.md)了解更多信息。 |\n| `repetition_penalty` | `float` | 如果您发现模型经常重复自己，请将其设置为 `>1.0`。较大的值会施加更大的惩罚。 |\n| `eos_token_id` | `list[int]` | 将导致生成停止的token。默认值通常是好的，但您可以指定不同的token。 |

## 常见陷阱

下面的部分涵盖了您在文本生成过程中可能遇到的一些常见问题以及如何解决它们。

### 输出长度

[generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) 默认返回最多20个token，除非在模型的 [GenerationConfig](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig) 中另有指定。强烈建议使用 `max_new_tokens` 参数手动设置生成的token数量来控制输出长度。[仅解码器](https://hf.co/learn/nlp-course/chapter1/6?fw=pt)模型返回初始提示词以及生成的token。

In [ ]:
model_inputs = tokenizer([\"A sequence of numbers: 1, 2\"], return_tensors=\"pt\").to(\"cuda\")

**默认长度：**

In [ ]:
generated_ids = model.generate(**model_inputs)\ntokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]\n# 输出: 'A sequence of numbers: 1, 2, 3, 4, 5'

**使用 max_new_tokens：**

In [ ]:
generated_ids = model.generate(**model_inputs, max_new_tokens=50)\ntokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]\n# 输出: 'A sequence of numbers: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,'

### 解码策略

[generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) 中的默认解码策略是*贪心搜索*，它选择下一个最可能的token，除非在模型的 [GenerationConfig](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig) 中另有指定。虽然这种解码策略在基于输入的任务（转录、翻译）上表现良好，但对于更具创意的用例（故事写作、聊天应用）来说并不是最优的。\n\n例如，启用[多项式采样](https://huggingface.co/docs/transformers/main/en/./generation_strategies#multinomial-sampling)策略来生成更多样化的输出。请参考[生成策略](https://huggingface.co/docs/transformers/main/en/./generation_strategies)指南了解更多解码策略。

In [ ]:
model_inputs = tokenizer([\"I am a cat.\"], return_tensors=\"pt\").to(\"cuda\")\n\n# 贪心搜索\ngenerated_ids = model.generate(**model_inputs)\nprint(\"贪心搜索:\", tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])\n\n# 多项式采样\ngenerated_ids = model.generate(**model_inputs, do_sample=True)\nprint(\"多项式采样:\", tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

### 填充方向

如果输入长度不同，需要进行填充。但是大语言模型没有被训练从填充token继续生成，这意味着 `padding_side()` 参数需要设置为输入的左侧。

In [ ]:
# 右填充（错误）\ntokenizer_right = AutoTokenizer.from_pretrained(\"mistralai/Mistral-7B-v0.1\", padding_side=\"right\")\nmodel_inputs = tokenizer_right([\"1, 2, 3\", \"A, B, C, D, E\"], padding=True, return_tensors=\"pt\").to(\"cuda\")\ngenerated_ids = model.generate(**model_inputs)\nprint(\"右填充结果:\", tokenizer_right.batch_decode(generated_ids, skip_special_tokens=True)[0])\n\n# 左填充（正确）\ntokenizer_left = AutoTokenizer.from_pretrained(\"mistralai/Mistral-7B-v0.1\", padding_side=\"left\")\ntokenizer_left.pad_token = tokenizer_left.eos_token\nmodel_inputs = tokenizer_left([\"1, 2, 3\", \"A, B, C, D, E\"], padding=True, return_tensors=\"pt\").to(\"cuda\")\ngenerated_ids = model.generate(**model_inputs)\nprint(\"左填充结果:\", tokenizer_left.batch_decode(generated_ids, skip_special_tokens=True)[0])

### 提示词格式

某些模型和任务期望特定的输入提示词格式，如果格式不正确，模型会返回次优的输出。您可以在[提示词工程](https://huggingface.co/docs/transformers/main/en/./tasks/prompting)指南中了解更多关于提示词的信息。\n\n例如，聊天模型期望输入为[聊天模板](https://huggingface.co/docs/transformers/main/en/./chat_templating)。您的提示词应包含 `role` 和 `content` 来指示谁在参与对话。如果您尝试将提示词作为单个字符串传递，模型并不总是返回预期的输出。

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM\n\ntokenizer = AutoTokenizer.from_pretrained(\"HuggingFaceH4/zephyr-7b-alpha\")\nmodel = AutoModelForCausalLM.from_pretrained(\"HuggingFaceH4/zephyr-7b-alpha\", device_map=\"auto\", load_in_4bit=True)\n\n# 无格式\nprompt = \"How many cats does it take to change a light bulb? Reply as a pirate.\"\nmodel_inputs = tokenizer([prompt], return_tensors=\"pt\").to(\"cuda\")\ninput_length = model_inputs.input_ids.shape[1]\ngenerated_ids = model.generate(**model_inputs, max_new_tokens=50)\nprint(\"无格式:\", tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])\n\n# 聊天模板\nmessages = [\n    {\"role\": \"system\", \"content\": \"You are a friendly chatbot who always responds in the style of a pirate\"},\n    {\"role\": \"user\", \"content\": \"How many cats does it take to change a light bulb?\"}\n]\nmodel_inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors=\"pt\").to(\"cuda\")\ninput_length = model_inputs.shape[1]\ngenerated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=50)\nprint(\"聊天模板:\", tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])

## 资源

请查看下面一些更具体和专业的文本生成库。\n\n- [Optimum](https://github.com/huggingface/optimum)：Transformers 的扩展，专注于在特定硬件设备上优化训练和推理\n- [Outlines](https://github.com/dottxt-ai/outlines)：用于约束文本生成的库（例如生成 JSON 文件）。\n- [SynCode](https://github.com/uiuc-focal-lab/syncode)：用于上下文无关语法引导生成的库（JSON、SQL、Python）。\n- [Text Generation Inference](https://github.com/huggingface/text-generation-inference)：用于大语言模型的生产就绪服务器。\n- [Text generation web UI](https://github.com/oobabooga/text-generation-webui)：用于文本生成的 Gradio Web UI。\n- [logits-processor-zoo](https://github.com/NVIDIA/logits-processor-zoo)：用于控制文本生成的额外 logits 处理器。